In [4]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np

### grid:

1x  |  xx  |  xx  |  xx  |  xx \
xx  |  9x  |  x3  |  xx  |  xx \
xx  |  xx  |  xx  |  x4  |  x5 

10x  |  x80   |  xx   |  xx   |  xx \
xx   |  xx  |  x30  |  xx   |  xx \
xx   |  xx   |  xx   |  x40  |  x50 

In [245]:
hsamp = np.array([[[[[1,10],[0,80],[0,6],[0,6],[0,6]],
                [[0,6],[9,6],[0,6],[0,6],[0,6]],
                [[0,6],[0,6],[0,6],[0,6],[0,6]]]],

                [[[[0,6],[0,6],[0,6],[0,6],[0,6]],
                [[0,6],[0,6],[3,30],[0,6],[0,6]],
                [[0,6],[0,6],[0,6],[4,40],[5,50]]]]])
bgnum = np.full_like(hsamp, 100)

print(hsamp.shape)
print(bgnum.shape)
# print(grid)
hsone=hsamp[...,0]
print(hsone.shape)

(2, 1, 3, 5, 2)
(2, 1, 3, 5, 2)
(2, 1, 3, 5)


In [246]:
repped = np.repeat(hsone[...,np.newaxis],2,axis=4)
print(repped.shape)

(2, 1, 3, 5, 2)


In [7]:
# one real
hsmax = np.amax(hsone, axis=(0,1,2))
shape = hsone.shape
print(shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3])
print(newshape)
hsone = hsone.reshape(newshape) # change hsamp to shape (M-1*Q-1*Z-1, F-1)
argmax = np.argmax(hsone, axis=0) # max at each frequency & realization
hsone = hsone.reshape(shape) # restore hsamp shape to (M-1, Q-1, Z-1, F-1)
print(hsone.shape)
mqz = np.array(np.unravel_index(argmax, shape[:-1])) # unravel indices
print('mqz', mqz.shape)
print(mqz)
f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
print(np.append(mqz, f_ids))
ssnew = np.append(mqz, f_ids).reshape(4,len(f_ids))
print('ssnew', ssnew)
# print(hsone[ssnew[0], ssnew[1], ssnew[2], ssnew[3]])

(2, 1, 3, 5)
(6, 5)
(2, 1, 3, 5)
mqz (3, 5)
[[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]]
[0 0 1 1 1 0 0 0 0 0 0 1 1 2 2 0 1 2 3 4]
ssnew [[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]
 [0 1 2 3 4]]


In [217]:
#hsamp for all bins at all realizations

#gets max for each frequency and realization
max = np.amax(hsamp, axis=(0,1,2))
print(max)
shape = hsamp.shape
print('shape',shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
print('newshape', newshape)
hsamp = hsamp.reshape(newshape)
argmax = np.argmax(hsamp, axis=0)
hsamp = hsamp.reshape(shape)

# shape (3, F, R)
mqz = np.array(np.unravel_index(argmax, shape[:-2])) # unravel indices
print('mqz', mqz.shape)


# I GOt IT!!!!!
# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append

f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
ssnew = np.append(mqz,f_ids, axis=0)
print(ssnew, ssnew.shape)

[[ 1 10]
 [ 9 80]
 [ 3 30]
 [ 4 40]
 [ 5 50]]
shape (2, 1, 3, 5, 2)
newshape (6, 5, 2)
mqz (3, 5, 2)
[[[0 0]
  [0 0]
  [1 1]
  [1 1]
  [1 1]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [1 0]
  [1 1]
  [2 2]
  [2 2]]

 [[0 0]
  [1 1]
  [2 2]
  [3 3]
  [4 4]]] (4, 5, 2)


In [184]:
# print(hsamp[ssnew[0,:,0], ssnew[1,:,0], ssnew[2,:,0], 
#         ssnew[3,:,0]][...,0])
# print(hsamp[ssnew[0,:,1], ssnew[1,:,1], ssnew[2,:,1], 
#         ssnew[3,:,1]][...,1])
print(hsamp.shape)
print(ssnew.shape)
ampmax=hsamp[ssnew[0,], ssnew[1], ssnew[2], 
        ssnew[3]].squeeze()
ampmax2 = hsamp[tuple(ssnew)]
# why does this give me duplicates??
print(ampmax.shape)
print(ampmax)
print(max.shape)
print(max)

(2, 1, 3, 5, 2)
(4, 5, 2)
(5, 2, 2)
[[[ 1 10]
  [ 1 10]]

 [[ 9  6]
  [ 0 80]]

 [[ 3 30]
  [ 3 30]]

 [[ 4 40]
  [ 4 40]]

 [[ 5 50]
  [ 5 50]]]
(5, 2)
[[ 1 10]
 [ 9 80]
 [ 3 30]
 [ 4 40]
 [ 5 50]]


include r indices calculation

In [224]:
# include also r indices: 
# Only way to avoid for loops over realizations?
#hsamp for all bins at all realizations

#gets max for each frequency and realization
max = np.amax(hsamp, axis=(0,1,2))
print(max)
shape = hsamp.shape
print('shape',shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
print('newshape', newshape)
hsamp = hsamp.reshape(newshape)
argmax = np.argmax(hsamp, axis=0)
hsamp = hsamp.reshape(shape)

# shape (3, F, R)
mqz = np.array(np.unravel_index(argmax, shape[:-2])) # unravel indices
# print('mqz', mqz.shape)

# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append
# Need r_ids in shape (1,F,R) to append
R=shape[4]
F=shape[3]
print('R =', R, 'F =',F)
f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
r_ids = np.linspace(0,R-1,R).astype(int) # realization indices
print(r_ids)
r_ids = np.tile(r_ids, F).reshape(F,R)[np.newaxis,:]
print(r_ids, r_ids.shape)
ssnew = np.append(mqz,f_ids, axis=0)
ssnew = np.append(ssnew,r_ids, axis=0)
print(ssnew.shape)

[[ 1 10]
 [ 9 80]
 [ 3 30]
 [ 4 40]
 [ 5 50]]
shape (2, 1, 3, 5, 2)
newshape (6, 5, 2)
R = 2 F = 5
[0 1]
[[[0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]]] (1, 5, 2)
(5, 5, 2)


In [237]:
print(max.shape)

(5, 2)


In [225]:
print(hsamp[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]])

[[ 1 10]
 [ 9 80]
 [ 3 30]
 [ 4 40]
 [ 5 50]]


In [200]:
print(ssnew[0,0,0], ssnew[0,0,0].shape)
# mass index of all freqs, all reals
print(bgnum.shape)
print(bgnum[ssnew[0,0,0], ssnew[1,0,0], ssnew[2,0,0], ssnew[3,0,0],0])

0 ()
(2, 1, 3, 5, 2)
100


bgnum

In [235]:
r=1
print(bgnum[...,r], bgnum.shape)
print(hsamp.shape)
print(hsamp[ssnew[0], ssnew[1], ssnew[2], 
        ssnew[3],ssnew[4]][...,r])
bgnumcop = np.copy(bgnum)
# bgnumcop[ssnew[0,:,:], ssnew[1,:,:], ssnew[2,:,:], ssnew[3,:,:]]=9
bgnumcop[[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]]]-=1
print(bgnumcop[...,r], bgnumcop.shape)

[[[[100 100 100 100 100]
   [100 100 100 100 100]
   [100 100 100 100 100]]]


 [[[100 100 100 100 100]
   [100 100 100 100 100]
   [100 100 100 100 100]]]] (2, 1, 3, 5, 2)
(2, 1, 3, 5, 2)
[10 80 30 40 50]
[[[[ 99  99 100 100 100]
   [100 100 100 100 100]
   [100 100 100 100 100]]]


 [[[100 100 100 100 100]
   [100 100  99 100 100]
   [100 100 100  99  99]]]] (2, 1, 3, 5, 2)


/var/folders/r0/rzjvt8tn7lqbr892wvvnr0j40000gn/T/ipykernel_28871/2073731583.py:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bgnumcop[[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]]]-=1


old arrs stuff

In [64]:
arr = np.array([[[1,1], [2,2], [3,3]]])
print(arr, arr.shape)
app = np.array([[[0], [5], [6]]])
print(app, app.shape)
res = np.append(arr, app, axis=2)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)

[[[1 1]
  [2 2]
  [3 3]]] (1, 3, 2)
[[[0]
  [5]
  [6]]] (1, 3, 1)
[[[1 1 0]
  [2 2 5]
  [3 3 6]]] (1, 3, 3)


In [81]:
arr = np.array([[1],[2],[3]])
print(arr, arr.shape)
app = np.array([[0],[0],[3]])
print(app, app.shape)
res = np.append(arr, app, axis=1)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

[[1]
 [2]
 [3]] (3, 1)
[[0]
 [0]
 [3]] (3, 1)
[[1 0]
 [2 0]
 [3 3]] (3, 2)


In [89]:
arr = np.array([[1,2],[3,4]])
print(arr, arr.shape)
app = np.array([[0,0]])
print(app, app.shape)
res = np.append(arr, app, axis=0)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

[[1 2]
 [3 4]] (2, 2)
[[0 0]] (1, 2)
[[1 2]
 [3 4]
 [0 0]] (3, 2)


In [86]:
arr = np.array([[1,2],[3,4]])
print(arr, arr.shape)
app = np.array([[0],[0]])
print(app, app.shape)
res = np.append(arr, app, axis=1)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

[[1 2]
 [3 4]] (2, 2)
[[0]
 [0]] (2, 1)
[[1 2 0]
 [3 4 0]] (2, 3)


In [101]:
arr = np.array([[[1],[2]],[[3],[4]],[[5],[6]]])
print(arr, arr.shape)
app = np.array([[[0],[0]]])
print(app, app.shape)
res = np.append(arr, app, axis=0)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

[[[1]
  [2]]

 [[3]
  [4]]

 [[5]
  [6]]] (3, 2, 1)
[[[0]
  [0]]] (1, 2, 1)
[[[1]
  [2]]

 [[3]
  [4]]

 [[5]
  [6]]

 [[0]
  [0]]] (4, 2, 1)


In [127]:
print(mqz.shape) # mqz is shape (3,5,2) = (3,F,R)
# I want to append frequencies to get shape (4,5,2) = (4,F,R)
# this means I need my f_indices in shape (1,5,2)
print(f_ids.shape)
print(f_ids[np.newaxis,:,np.newaxis].shape)
print(np.tile(f_ids, (1,2)))
print(np.repeat(f_ids, 2).reshape(5,2).shape)

(3, 5, 2)
(5,)
(1, 5, 1)
[[0 1 2 3 4 0 1 2 3 4]]
(5, 2)


In [131]:
R = 2
F = 5
reps = np.repeat(f_ids, R).reshape(F,R)
resh = reps[np.newaxis,:]
print(resh)
print(resh.shape)

[[[0 0]
  [1 1]
  [2 2]
  [3 3]
  [4 4]]]
(1, 5, 2)


f appending

In [136]:
# I GOt IT!!!!!
# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append

f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
mqzf = np.append(mqz,f_ids, axis=0)
print(mqzf.shape)

(4, 5, 2)


single realization

In [252]:
hsamp = np.array([[[[1,0,0,0,0],[0,9,0,0,0],[0,0,0,0,0]]],

                 [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])[...,np.newaxis]
print(hsamp.shape)
bgnum = np.full_like(hsamp, 100)


(2, 1, 3, 5, 1)


In [260]:
# include also r indices: 
# Only way to avoid for loops over realizations?
#hsamp for all bins at all realizations

#gets max for each frequency and realization
max = np.amax(hsamp, axis=(0,1,2))
print(max)
shape = hsamp.shape
print('shape',shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
print('newshape', newshape)
hsamp = hsamp.reshape(newshape)
argmax = np.argmax(hsamp, axis=0)
hsamp = hsamp.reshape(shape)

# shape (3, F, R)
mqz = np.array(np.unravel_index(argmax, shape[:-2])) # unravel indices
# print('mqz', mqz.shape)

# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append
# Need r_ids in shape (1,F,R) to append
R=shape[4]
F=shape[3]
print('R =', R, 'F =',F)
f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
print('f_ids',f_ids, f_ids.shape)
r_ids = np.linspace(0,R-1,R).astype(int) # realization indices
print('r_ids',r_ids,r_ids.shape)
r_ids = np.tile(r_ids, F).reshape(F,R)[np.newaxis,:]
print(r_ids, r_ids.shape)
ssnew = np.append(mqz,f_ids, axis=0)
ssnew = np.append(ssnew,r_ids, axis=0)
print(ssnew.shape)

[[1]
 [9]
 [3]
 [4]
 [5]]
shape (2, 1, 3, 5, 1)
newshape (6, 5, 1)
R = 1 F = 5
f_ids [[[0]
  [1]
  [2]
  [3]
  [4]]] (1, 5, 1)
r_ids [0] (1,)
[[[0]
  [0]
  [0]
  [0]
  [0]]] (1, 5, 1)
(5, 5, 1)


In [257]:
print(ssnew)

[[[0]
  [0]
  [1]
  [1]
  [1]]

 [[0]
  [0]
  [0]
  [0]
  [0]]

 [[0]
  [1]
  [1]
  [2]
  [2]]

 [[0]
  [1]
  [2]
  [3]
  [4]]

 [[0]
  [0]
  [0]
  [0]
  [0]]]


In [256]:
r=0
print(bgnum[...,r], bgnum.shape)
print(hsamp.shape)
print(hsamp[ssnew[0], ssnew[1], ssnew[2], 
        ssnew[3],ssnew[4]][...,r])
bgnumcop = np.copy(bgnum)
# bgnumcop[ssnew[0,:,:], ssnew[1,:,:], ssnew[2,:,:], ssnew[3,:,:]]=9
bgnumcop[[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]]]-=1
print(bgnumcop, bgnumcop.shape)

[[[[100 100 100 100 100]
   [100 100 100 100 100]
   [100 100 100 100 100]]]


 [[[100 100 100 100 100]
   [100 100 100 100 100]
   [100 100 100 100 100]]]] (2, 1, 3, 5, 1)
(2, 1, 3, 5, 1)
[1 9 3 4 5]
[[[[[ 99]
    [100]
    [100]
    [100]
    [100]]

   [[100]
    [ 99]
    [100]
    [100]
    [100]]

   [[100]
    [100]
    [100]
    [100]
    [100]]]]



 [[[[100]
    [100]
    [100]
    [100]
    [100]]

   [[100]
    [100]
    [ 99]
    [100]
    [100]]

   [[100]
    [100]
    [100]
    [ 99]
    [ 99]]]]] (2, 1, 3, 5, 1)


/var/folders/r0/rzjvt8tn7lqbr892wvvnr0j40000gn/T/ipykernel_28871/70686756.py:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bgnumcop[[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]]]-=1


# From ss_real_dev 3/10

## Scratch

grid:

1x  |  xx  |  xx  |  xx  |  xx \
xx  |  9x  |  x3  |  xx  |  xx \
xx  |  xx  |  xx  |  x4  |  x5 

In [ ]:
grid = np.array([[[[1,0,0,0,0],[0,9,0,0,0],[0,0,0,0,0]]],
                 [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])
print(grid.shape)
mqz_f, f_mqz, all_f, f_all= max_index_at_f(grid)
print('mqz_f:\n',mqz_f)
print('\nf_mqz:\n', f_mqz)
print('\nall_f:\n', all_f)
print('\nf_all:\n', f_all)

(2, 1, 3, 5)
mqz_f:
 [[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]]

f_mqz:
 [[0 0 0]
 [0 0 1]
 [1 0 1]
 [1 0 2]
 [1 0 2]]

all_f:
 [[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]
 [0 1 2 3 4]]

f_all:
 [[0 0 0 0]
 [0 0 1 1]
 [1 0 1 2]
 [1 0 2 3]
 [1 0 2 4]]


In [ ]:
maxes = np.amax(grid, axis=(0,1,2))
print(maxes)

hsmax = np.amax(grid, axis=(0,1,2)) #find max hs at each frequency
    
    #### 2) Record the indices and strain of that single source

# --- Indices of Loudest Bin ---
# Shape [F, 4], looks like
# [[m_idx,q_idx,z_idx,0],
#  [m_idx,q_idx,z_idx,1],
#   ........
#  [m_idx,q_idx,z_idx,F-2]]
ssidx = np.argwhere(grid==hsmax) 
for s in range(len(ssidx)):
    m,q,z,f = ssidx[s]
    print(m,q,z,f, 'grid max:',grid[m,q,z,f])
print(ssidx)

[1 9 3 4 5]
0 0 0 0 grid max: 1
0 0 1 1 grid max: 9
1 0 1 2 grid max: 3
1 0 2 3 grid max: 4
1 0 2 4 grid max: 5
[[0 0 0 0]
 [0 0 1 1]
 [1 0 1 2]
 [1 0 2 3]
 [1 0 2 4]]


grid: argwhere vs where

In [ ]:
where = np.where(grid==hsmax)
where_arr = np.array([*where])
argwhere = np.argwhere(grid==hsmax)
print('where:\n', where)
print('np.where unpacked and put in numpy array\n', where_arr)
print('argwhere:\n', argwhere)
print('first point', argwhere[0,:], where_arr[:,0])
for f in range(len(argwhere)):
    print('f: by np.array[*where][:,f]', where_arr[:,f],  
          'or by argwhere[f,:]', argwhere[f,:])

where:
 (array([0, 0, 1, 1, 1]), array([0, 0, 0, 0, 0]), array([0, 1, 1, 2, 2]), array([0, 1, 2, 3, 4]))
np.where unpacked and put in numpy array
 [[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]
 [0 1 2 3 4]]
argwhere:
 [[0 0 0 0]
 [0 0 1 1]
 [1 0 1 2]
 [1 0 2 3]
 [1 0 2 4]]
first point [0 0 0 0] [0 0 0 0]
f: by np.array[*where][:,f] [0 0 0 0] or by argwhere[f,:] [0 0 0 0]
f: by np.array[*where][:,f] [0 0 1 1] or by argwhere[f,:] [0 0 1 1]
f: by np.array[*where][:,f] [1 0 1 2] or by argwhere[f,:] [1 0 1 2]
f: by np.array[*where][:,f] [1 0 2 3] or by argwhere[f,:] [1 0 2 3]
f: by np.array[*where][:,f] [1 0 2 4] or by argwhere[f,:] [1 0 2 4]


grid: using reshape

In [ ]:
shape = grid.shape
print(shape)
print(grid)
grid = grid.reshape(6,5)
print(grid.shape)
print(grid)
grid = grid.reshape(shape)
print(grid.shape)
print(grid)

(2, 1, 3, 5)
[[[[1 0 0 0 0]
   [0 9 0 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 3 0 0]
   [0 0 0 4 5]]]]
(6, 5)
[[1 0 0 0 0]
 [0 9 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 5]]
(2, 1, 3, 5)
[[[[1 0 0 0 0]
   [0 9 0 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 3 0 0]
   [0 0 0 4 5]]]]


In [ ]:
shape = (2,1,3,5)
print(shape[:-1])
grid = grid.reshape(shape)
# print('grid:', grid.shape,'\n', grid)
grid = grid.reshape(6,5)
print('grid:', grid.shape,'\n', grid)
# print('grid[4,2]', grid[4,2])

argmax0 = np.argmax(grid,axis=0)
# argmax1 = np.argmax(grid,axis=1)
# unraveledargmax = np.unravel_index(argmax, grid.shape)
print('argmax0:', argmax0.shape, argmax0)
for i in range(len(grid[0])):
    print('argmax[i]=',argmax0[i], 
          'i=', i, 
          'grid[argmax[i], i]] =', grid[argmax0[i], i])
# print('argmax1:', argmax1.shape, argmax1)

# print('unraveled:', unraveledargmax)
# print('max at argmax:', grid[:,argmax])
# argidx = np.unravel_index(argmax, grid.shape)
print('grid[argmax,:]\n', grid[argmax0,:])
# print('grid[:,argmax]\n', grid[:,argmax0])
grid = grid.reshape(shape)
unraveled = np.unravel_index(argmax0, shape[:-1])
print('unraveled', len(unraveled), unraveled[0].shape, unraveled)
for i in range(len(grid[0,0,0,:])):
    m,q,z= unraveled[0][i], unraveled[1][i], unraveled[2][i] #, unraveled[3][i]
    print('unraveled[:,i] m,q,z,f = ', m,q,z,i)
    print(grid[m,q,z,i])
    # print('argmax[i]=',argmax0[i], 
    #       'i=', i, 
    #       'grid[argmax[i], i]] =', grid[n,q,z, i])

# print('grid:', grid.shape,'\n', grid)



(2, 1, 3)
grid: (6, 5) 
 [[1 0 0 0 0]
 [0 9 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 5]]
argmax0: (5,) [0 1 4 5 5]
argmax[i]= 0 i= 0 grid[argmax[i], i]] = 1
argmax[i]= 1 i= 1 grid[argmax[i], i]] = 9
argmax[i]= 4 i= 2 grid[argmax[i], i]] = 3
argmax[i]= 5 i= 3 grid[argmax[i], i]] = 4
argmax[i]= 5 i= 4 grid[argmax[i], i]] = 5
grid[argmax,:]
 [[1 0 0 0 0]
 [0 9 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 5]
 [0 0 0 4 5]]
unraveled 3 (5,) (array([0, 0, 1, 1, 1]), array([0, 0, 0, 0, 0]), array([0, 1, 1, 2, 2]))
unraveled[:,i] m,q,z,f =  0 0 0 0
1
unraveled[:,i] m,q,z,f =  0 0 1 1
9
unraveled[:,i] m,q,z,f =  1 0 1 2
3
unraveled[:,i] m,q,z,f =  1 0 2 3
4
unraveled[:,i] m,q,z,f =  1 0 2 4
5


figured out how to use my faster mqz indexing to get maxes from grid, with fancy indexing

In [ ]:
mqz = reshaped_argmax_at_f(grid)
print(mqz)
mqzf = mqzf_from_mqz(mqz, True)
print(mqzf)

[[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]]
f_indices [0 1 2 3 4]
m,q,z,f 0 0 0 0
grid[m,q,z,f] = 1
m,q,z,f 0 0 1 1
grid[m,q,z,f] = 9
m,q,z,f 1 0 1 2
grid[m,q,z,f] = 3
m,q,z,f 1 0 2 3
grid[m,q,z,f] = 4
m,q,z,f 1 0 2 4
grid[m,q,z,f] = 5
mqzf (5, 4) 
 [[0 0 0 0]
 [0 0 1 1]
 [1 0 1 2]
 [1 0 2 3]
 [1 0 2 4]]
[[0 0 0 0]
 [0 0 1 1]
 [1 0 1 2]
 [1 0 2 3]
 [1 0 2 4]]


In [ ]:
faster = faster_mqzf_from_mqz(mqz, False)
print('faster:', faster)
print(grid.shape)
print('1st max at m,q,z,f = 0,0,0,0:', grid[0,0,0,0])
print('2nd max at m,q,z,f = 0,0,1,1:', grid[0,0,1,1])
print('3rd max at m,q,z,f = 1,0,1,2:', grid[1,0,1,2])
print('4th max at m,q,z,f = 1,0,2,3:', grid[1,0,2,3])
print('5th max at m,q,z,f = 1,0,2,4:', grid[1,0,2,4])
print(grid[[0,0,1,1,1], [0,0,0,0,0], [0,1,1,2,2], [0,1,2,3,4]])
fancy = [0,0,1,1,1], [0,0,0,0,0], [0,1,1,2,2], [0,1,2,3,4]
print('grid[fancy] =',grid[fancy])
print('fancy:', fancy)
print(grid[faster[0], faster[1], faster[2], faster[3]]) # this gives maxes!!! 

# listed = (faster[0], faster[1], faster[2], faster[3])
# print(listed)


faster: [[0 0 1 1 1]
 [0 0 0 0 0]
 [0 1 1 2 2]
 [0 1 2 3 4]]
(2, 1, 3, 5)
1st max at m,q,z,f = 0,0,0,0: 1
2nd max at m,q,z,f = 0,0,1,1: 9
3rd max at m,q,z,f = 1,0,1,2: 3
4th max at m,q,z,f = 1,0,2,3: 4
5th max at m,q,z,f = 1,0,2,4: 5
[1 9 3 4 5]
grid[fancy] = [1 9 3 4 5]
fancy: ([0, 0, 1, 1, 1], [0, 0, 0, 0, 0], [0, 1, 1, 2, 2], [0, 1, 2, 3, 4])
[1 9 3 4 5]


using reshaped argmax, and mqzf to mqz methods

In [ ]:
# this is what mqz looks like
# all [[M indices], [q indices], [z indices]] in shape (3,F)
arr = np.array([[1,2,3,4,5], [10,20,30,40,50], [100,200,300,400,500]])
print(np.array(arr).shape)
# Here are the F indices
indices = np.array([0,1,2,3,4])

# I'd like to get this in the shape (4,F):
# [array of M indices, array of q indices, array of z indices]
newarr = np.append(arr,indices).reshape(4,len(indices))
print(newarr)

(3, 5)
[[  1   2   3   4   5]
 [ 10  20  30  40  50]
 [100 200 300 400 500]
 [  0   1   2   3   4]]


apply to example 3

In [ ]:
edges, number, fobs, exname = example2(print_test=False)
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum, ssnew = ss_by_ndars(edges, number, realize=False, round=True)
# print(holo.utils.stats(hsamp))
np.set_printoptions(precision=2)
print('hsamp\n', hsamp)
print('hsmax', hsmax)
shape = hsamp.shape



ValueError: not enough values to unpack (expected 7, got 6)

In [ ]:
print('old hsamp', hsamp.shape, '\n', hsamp)

newshape = (shape[0]*shape[1]*shape[2], shape[3])
hsamp = hsamp.reshape(newshape)
print('new hsamp', hsamp.shape, '\n', hsamp)
argmax = np.argmax(hsamp, axis=0)
print('argmax', argmax)


hsamp = hsamp.reshape(shape)
print('m,q,z shape:', shape[:-1])
maxidx = np.unravel_index(argmax, shape[:-1])

for i in range(len(hsamp[0,0,0,:])):
    m,q,z= maxidx[0][i], maxidx[1][i], maxidx[2][i] #, unraveled[3][i]
    print('unraveled[:,i] m,q,z,f = ', m,q,z,i)
    print(hsamp[m,q,z,i], hsmax[i])


old hsamp (2, 1, 3, 5, 2) 
 [[[[[0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]]

   [[3.04824418e-20 3.04824418e-20]
    [4.83878602e-20 4.83878602e-20]
    [6.34060341e-20 6.34060341e-20]
    [7.68109401e-20 7.68109401e-20]
    [9.20780165e-20 9.20780165e-20]]

   [[4.45214787e-21 4.45214787e-21]
    [7.06734422e-21 7.06734422e-21]
    [9.26084077e-21 9.26084077e-21]
    [1.12187096e-20 1.12187096e-20]
    [1.34485599e-20 1.34485599e-20]]]]



 [[[[0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]
    [0.00000000e+00 0.00000000e+00]]

   [[1.41486961e-18 1.41486961e-18]
    [2.24596551e-18 2.24596551e-18]
    [2.94304740e-18 2.94304740e-18]
    [3.56524802e-18 3.56524802e-18]
    [4.27388293e-18 4.27388293e-18]]

   [[2.06650399e-19 2.06650399e-19]
    [3.28037060e-19 3.

ValueError: cannot reshape array of size 60 into shape (6,5)

In [ ]:
# am = (np.argmax(hsamp[3,3,:,4]))
# print(hsamp[3,3,:,4])
# print(hsamp[3,3,am,4])

In [ ]:
# mqz_f, f_mqz, all_f, f_all= max_index_at_f(grid)
# argwhere = argwhere_at_f(grid)
# print(argwhere)
# assert (np.all(f_all == argwhere)), 'argwhere failing to find maxes'

In [ ]:
# for i in range(len(grid[0,0,0,:])):
#     m,q,z,f = argwhere[i]
#     print(m,q,z,f, 'max:', grid[m,q,z,f])

In [ ]:
# grid = np.array([[[[1,0,0,0,0],[0,2,0,0,0],[0,0,0,0,0]]],
#                  [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])
# maxes = np.max(grid, axis=(0,1,2))
# # print('grid:', grid.shape, '\n', grid)
# # print('maxes', maxes.shape, '\n', maxes)
# where = np.where(grid==maxes)
# print('where: len=', len(where), 'with shapes=', where[0].shape, where[1].shape, where[2].shape,
#       where[3].shape, '\n', where)
# print('max by where:', grid[np.where(maxes==grid)])
# print()
# argmax = np.argmax(grid)
# m,q,z,f = np.unravel_index(np.argmax(grid), grid.shape)
# print('m:',m, '\nq:',q, '\nz:',z, '\nf:',f)
# print('argmaxes:', argmax.shape, '\n', argmax)

In [ ]:
# idxs = np.argwhere(arr==maxes)
# print('idxs', idxs.shape,'\n', idxs)
# rots = np.rot90(idxs)
# print('rots', rots.shape, '\n', rots)
# print(idxs[0])
# for i in [0,1]:
#     print(arr[idxs[i,0], idxs[i,1], idxs[i,2]])
# for i in [0,1]:
#     # print(arr[rots[0,i], rots[1,i], rots[2,i]])
#     print(arr[rots[2,i], rots[1,i], rots[0,i]])
# # the rot situation is trickier, just use
# # idxs = np.argwhere(arr==maxes)
# # returns (2,3) array
# # or (f, 4) array?


# Cython


In [1]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy

reload(cy) # not working :/
reload(ss)

<module 'holodeck.single_sources' from '/Users/emigardiner/GWs/holodeck/holodeck/single_sources.py'>

Using SS's

In [2]:
edges, number, fobs, exname = ss.example2(print_test=False)
h2fdf = ss.h2fdf(edges)

13:30:01 INFO : zero_coalesced=True, zero_stalled=False [sam.py:dynamic_binary_number]
13:30:01 INFO : Stalled SAM bins based on GMT: 1.20e+01/2.40e+01 = 5.00e-01 [sam.py:static_binary_density]
13:30:01 INFO : Adding MMbulge scatter (3.4000e-01) [sam.py:static_binary_density]
13:30:01 INFO : 	dens bef: (1.79e-03, 1.94e-03, 4.00e-03, 7.66e-03, 1.48e-02, 1.53e-02, 1.54e-02) [sam.py:static_binary_density]
13:30:01 INFO : Scatter added after 0.002437 sec [sam.py:static_binary_density]
13:30:01 INFO : 	dens aft: (2.07e-03, 2.12e-03, 4.31e-03, 7.29e-03, 1.38e-02, 1.46e-02, 1.46e-02) [sam.py:static_binary_density]
13:30:01 INFO : 	mass: 1.21e-02 ==> 1.20e-02 || change = -9.0094e-03 [sam.py:static_binary_density]
13:30:01 INFO : zeroing out 1.20e+01/2.40e+01 = 5.00e-01 systems stalled from GMT [sam.py:static_binary_density]
13:30:01 INFO : fraction of coalesced binaries: 0.00e+00/1.20e+02 = 0.00e+00 [sam.py:dynamic_binary_number]
13:30:01 WARNING : WARNING: _stalled_ binaries are not being acc

In [3]:
hc_ss, hc_bg, ssidx = cy.ss_bg_hc(number, h2fdf, 2)

[[2.00185603e-36 2.00185603e-36]
 [1.00887222e-35 1.00887222e-35]
 [2.59845839e-35 2.59845839e-35]
 [5.08439738e-35 5.08439738e-35]
 [8.66611127e-35 8.66611127e-35]] [[8.41801361e-31 8.41881442e-31]
 [3.11013465e-31 3.12286731e-31]
 [1.78109026e-31 1.81235363e-31]
 [1.22007831e-31 1.21111094e-31]
 [8.86345056e-32 9.06408785e-32]] [[[1 1]
  [1 1]
  [1 1]
  [1 1]
  [1 1]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]
  [0 0]]

 [[1 1]
  [1 1]
  [1 1]
  [1 1]
  [1 1]]]


Basic Tests

In [2]:
cy.test_long_array(5)

[2 2 2 2 2]


array([2, 2, 2, 2, 2])

In [2]:
arr1, arr2 = cy.test_multiple_arrays()

[[0. 0. 0.]
 [0. 0. 0.]] [[0. 0. 0.]
 [0. 0. 0.]]
[[1. 1. 0.]
 [1. 1. 0.]] [[2. 2. 0.]
 [2. 2. 0.]]


In [2]:
arr = cy.test_2darray()

[[0. 0. 0.]
 [0. 0. 0.]]
[[6.2 6.2 6.2]
 [6.2 6.2 6.2]]


In [4]:
arr = cy.test_1darray()
print(arr)

[0. 0. 0. 0. 0.]
[5. 5. 5. 5. 5.]
[5. 5. 5. 5. 5.]
